In [1]:
from sklearn import metrics, random_projection, cross_validation, datasets, grid_search, linear_model, metrics, random_projection, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from scipy import load, save
import sklearn
%pylab inline

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [29]:
dataframe = pd.read_csv('dataframe/genres_40.csv')
dataframe.loc[:, 'm1':] = preprocessing.normalize(dataframe.loc[:, 'm1':])

labels = ['hiphop', 'neoclassic', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
labels_n = ['neoclassic', 'jazz',  'metal', 'pop']

# dataframe = dataframe[dataframe['tag'] == 'neoclassic'].append(dataframe[dataframe['tag'] == 'jazz']).append(dataframe[dataframe['tag'] == 'pop']).append(dataframe[dataframe['tag'] == 'metal'])
# dataframe.append(dataframe[dataframe['tag'] == 'pop'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'jazz'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'metal'])
dataframe = dataframe.sample(frac=1)

shape(dataframe)

(1601, 201)

In [30]:
def search_abnom(dataframe, label, g, nu):
    svm_ = OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for label in labels:
    svm_label = search_abnom(dataframe, label, 2, 0.15).tolist()
    dataframe[dataframe['tag'] == label] = dataframe[dataframe['tag'] == label][np.array(svm_label) == 1]
    dataframe = dataframe.dropna()
print(shape(dataframe))

(1353, 201)


In [4]:
def max_cl(arr):
    list_max_i = []
    for a in arr:
        max_el = 0
        max_i = 0
        for i in range(len(a)):
            if a[i] > max_el:
                max_el = a[i]
                max_i = i
        list_max_i += [max_i]
    return list_max_i

max_cl([[1,2,3], [2,6,4], [8,1,2]])

[2, 1, 0]

In [5]:
label_dict = {'hiphop':0, 'classical':1, 'jazz':2, 'rock':3, 'metal':4, 'synth':5, 'pop':6, 'blues': 7} 
int_label = []
for label in dataframe['tag']:
    int_label.append(label_dict[label])

In [6]:
# svm_label = []
# for label in labels:
#     svm_label += search_abnom(label, 2, 0.01).tolist()
# dataframe_svm = dataframe[np.array(svm_label) == 1]

# dataframe_svm_t = np.array(dataframe_svm.loc[:, 'm1':])
# dataframe_svm_l = np.array(dataframe_svm['tag'])

dataframe_svm_t = np.array(dataframe.loc[:, 'm1':])
dataframe_svm_l = np.array(int_label)
np.shape(dataframe_svm_l)

(1355,)

In [7]:
dataframe_t = dataframe.loc[:, 'm1':]

In [19]:
svcc = SVC(C=10, gamma=5, kernel = 'rbf', probability=True, random_state=10)
lrc = LogisticRegression(C = 430, random_state = 15)
knc = KNeighborsClassifier(weights = 'distance', n_neighbors=7, metric = 'manhattan')
rfc = RandomForestClassifier(max_depth=23, n_estimators=1300)
xgbc = xgb.XGBClassifier(silent=False, nthread=4, max_depth=2, n_estimators=4000, learning_rate=0.009)

In [42]:
def fin_classif(list_model, list_param):
    res_list = []
    for train_ind, test_ind in cross_validation.StratifiedKFold(dataframe_svm_l, n_folds=5, shuffle=True, random_state=1):
        np.random.shuffle(train_ind) 
        np.random.shuffle(test_ind)

#         f_df = dataframe_svm_t[:, :]
#         s_df = dataframe_svm_t[:, :]
        df_ = dataframe_svm_t[:, list_param[0]]
        svcc.fit(df_[train_ind], dataframe_svm_l[train_ind])
        list_preds = list_model[0].predict_proba(df_[test_ind])

        for i in range(1, len(list_model)):
            df_ = dataframe_svm_t[:, list_param[i]]
            list_model[i].fit(df_[train_ind], dataframe_svm_l[train_ind])
            list_preds += list_model[i].predict_proba(df_[test_ind])
#             list_preds += list_model[i].predict_proba(dataframe_svm_t[test_ind])
            
        res = max_cl(list_preds)
        
        res = metrics.accuracy_score(dataframe_svm_l[test_ind], res)
        res_list += [res]
        print(res)
    return res_list

In [41]:
rl = fin_classif([svcc, xgbc], [load('regr_param.npy'), load('boost_param.npy')])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

(1355, 165)
0.8613138686131386
(1355, 165)
0.8713235294117647
(1355, 165)
0.8708487084870848
(1355, 165)
0.8962962962962963
(1355, 165)
0.8768656716417911
mean = 0.8753296148900152
std =  0.011614900656362233


In [12]:
rl = fin_classif([svcc, rfc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8613138686131386
0.8897058823529411
0.8929889298892989
0.8851851851851852
0.8731343283582089
mean = 0.8804656388797547
std =  0.011704934453393274


In [29]:
rl = fin_classif([lrc, knc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8487084870848709
0.8154981549815498
0.8666666666666667
0.8296296296296296
0.8458646616541353
mean = 0.8412735200033705
std =  0.01744201157933252


In [13]:
rl = fin_classif([lrc, knc, rfc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.843065693430657
0.8492647058823529
0.8265682656826568
0.8518518518518519
0.8470149253731343
mean = 0.8435530884441306
std =  0.008968813978657248


In [14]:
rl = fin_classif([knc, lrc, svcc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8613138686131386
0.8823529411764706
0.8597785977859779
0.8777777777777778
0.8694029850746269
mean = 0.8701252340855984
std =  0.008868967623129654


In [15]:
rl = fin_classif([rfc, knc, svcc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8722627737226277
0.8897058823529411
0.8782287822878229
0.8888888888888888
0.8694029850746269
mean = 0.8796978624653816
std =  0.008343364674503799


In [16]:
rl = fin_classif([lrc, rfc, knc, svcc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8540145985401459
0.8602941176470589
0.8708487084870848
0.8740740740740741
0.8768656716417911
mean = 0.867219434078031
std =  0.008666433067874553


In [17]:
rl = fin_classif([lrc, rfc, svcc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8467153284671532
0.8455882352941176
0.8560885608856088
0.8629629629629629
0.8656716417910447
mean = 0.8554053458801775
std =  0.008183804168487158


In [18]:
rl = fin_classif([svcc])
print('mean =', np.mean(rl))
print('std = ', np.std(rl))

0.8540145985401459
0.8786764705882353
0.8819188191881919
0.8777777777777778
0.8656716417910447
mean = 0.871611861577079
std =  0.010383741866680165
